#Setup

In [ ]:
!pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 14.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import gurobipy as gp
import pandas as pd

In [ ]:
#Read in the files
df1 = pd.read_csv('ROI_data.csv')
df2 = pd.read_csv('roi_mat.csv')
df3 = pd.read_csv('index_data.csv')

In [ ]:
#Clean the dataset
df1.drop(['Platform'], axis = 1, inplace = True)

In [ ]:
df1.head()

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
0,0.031,0.049,0.024,0.039,0.016,0.024,0.046,0.026,0.033,0.044
1,0.049,0.023,0.024,0.039,0.044,0.046,0.026,0.019,0.037,0.026


In [ ]:
#Clean the dataset
df2.rename(columns = {'Unnamed: 0': 'Month'}, inplace = True)
df2.drop(['Month'], axis = 1, inplace = True)

In [ ]:
df2.head()

,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
0,4.0,3.6,2.4,3.9,3.0,3.5,3.6,2.25,3.5,3.5
1,4.0,3.9,2.7,3.8,4.3,3.2,2.7,1.80,3.7,3.5
2,3.5,2.9,3.1,3.8,2.4,4.1,3.7,2.60,4.2,2.5
3,3.8,3.1,2.4,4.4,2.4,3.8,3.7,2.50,3.6,2.9
4,3.5,3.2,1.9,3.4,2.7,2.7,3.9,2.20,4.5,3.9


#Q1-3

In [ ]:
Mod=gp.Model()
ModX = Mod.addMVar(10, ub = 3) # tell the model how many variables there are
ROI1 = np.array(df1.loc[0])
for index in range(len(ROI1)):
  ROI1[index] = ROI1[index]+1


# add the constraints to the model
conlist=[0]*3
conlist[0] = Mod.addConstr(ModX[0] + ModX[1] + ModX[2] + ModX[3] + ModX[4] + ModX[5] + ModX[6] + ModX[7] + ModX[8] + ModX[9] <= 10) 
conlist[1] = Mod.addConstr(1*ModX[0] + 1*ModX[1] - 1*ModX[4] - 1* ModX[9] <= 0)
conlist[2] = Mod.addConstr(-2*ModX[2] + -2*ModX[3] + 1* ModX[4] + 1*ModX[5] + 1*ModX[6] + 1*ModX[7] + 1*ModX[8] >= 0)
Mod.setObjective(ROI1 @ ModX, sense=gp.GRB.MAXIMIZE)


Mod.Params.OutputFlag = 0 # tell gurobi to shut up!!
Mod.Params.TimeLimit = 3600 # stop if taking too long
Mod.optimize() # solve the LP, will take the most time
print(Mod.objVal) # optimal ROI
print(Mod.x) # how much to invest in each investment

Restricted license - for non-production use only - expires 2024-10-28
10.456
[0.0, 3.0, 0.0, 1.0, 0.0, 0.0, 3.0, 0.0, 0.0, 3.0]


#Q4

In [ ]:
Mod2=gp.Model()
ModX2 = Mod2.addMVar(10, ub = 3) # tell the model how many variables there are
ROI2 = np.array(df1.loc[1])
for index in range(len(ROI2)):
  ROI2[index] = ROI2[index]+1


# add the constraints to the model
conlist2=[0]*3
conlist2[0] = Mod2.addConstr(ModX2[0] + ModX2[1] + ModX2[2] + ModX2[3] + ModX2[4] + ModX2[5] + ModX2[6] + ModX2[7] +ModX2[8] + ModX2[9] <= 10) 
conlist2[1] = Mod2.addConstr(1*ModX2[0] + 1*ModX2[1] - 1*ModX2[4] - 1* ModX2[9] <= 0)
conlist2[2] = Mod2.addConstr(-2*ModX2[2] + -2*ModX2[3] + 1* ModX2[4] + 1*ModX2[5] + 1*ModX2[6] + 1*ModX2[7] + 1*ModX2[8] >= 0)
Mod2.setObjective(ROI2 @ ModX2, sense=gp.GRB.MAXIMIZE)


Mod2.Params.OutputFlag = 0 # tell gurobi to shut up!!
Mod2.Params.TimeLimit = 3600 # stop if taking too long
Mod2.optimize() # solve the LP, will take the most time
print(Mod2.objVal) # optimal ROI
print(Mod2.x) # how much to invest in each investment

10.456
[3.0, 0.0, 0.0, 1.0, 3.0, 3.0, 0.0, 0.0, 0.0, 0.0]


#Q5

In [ ]:
#Using optimal allocation form 2 on the first ROI
incomeIfFirstCorrect = Mod2.X @ ROI1
print(incomeIfFirstCorrect, Mod.objVal)
incomeIfFirstCorrect - Mod.objVal

10.251999999999999 10.456


-0.20400000000000063

Q5 a) If the first ROI is correct and we use the allocation from the second model (with the second ROI), then we would lose 0.204 million dollars. 

In [ ]:
#Using optimal allocation form 1 on the second ROI
incomeIfSecondCorrect = Mod.X @ ROI2
print(incomeIfSecondCorrect, Mod2.objVal)
incomeIfSecondCorrect - Mod2.objVal

10.264 10.456


-0.19200000000000017

Q5 b) If the second ROI is correct and we use the allocation from the first model (with the first ROI), then we would lose 0.192 million dollars. 

In [ ]:
# Mod=gp.Model()
# ModX = Mod.addMVar(10) # tell the model how many variables there are
# ROI1 = np.array(df1.loc[0])
# for index in range(len(ROI1)):
#   ROI1[index] = ROI1[index]+1


# # add the constraints to the model
# conlist=[0]*3
# conlist[0] = Mod.addConstr(ModX[0] + ModX[1] + ModX[2] + ModX[3] + ModX[4] + ModX[5] + ModX[6] + ModX[7] + ModX[8] + ModX[9] <= 10) 
# conlist[1] = Mod.addConstr(1*ModX[0] + 1*ModX[1] - 1*ModX[4] - 1* ModX[9] <= 0)
# conlist[2] = Mod.addConstr(-2*ModX[2] + -2*ModX[3] + 1* ModX[4] + 1*ModX[5] + 1*ModX[6] + 1*ModX[7] + 1*ModX[8] >= 0)
# Mod.setObjective(ROI1 @ ModX, sense=gp.GRB.MAXIMIZE)


# Mod.Params.OutputFlag = 0 # tell gurobi to shut up!!
# Mod.Params.TimeLimit = 3600 # stop if taking too long
# Mod.optimize() # solve the LP, will take the most time
# print(Mod.objVal) # optimal ROI
# print(Mod.x) # how much to invest in each investment

10.465
[0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0]


In [ ]:
# Mod2=gp.Model()
# ModX2 = Mod2.addMVar(10) # tell the model how many variables there are
# ROI2 = np.array(df1.loc[1])
# for index in range(len(ROI2)):
#   ROI2[index] = ROI2[index]+1


# # add the constraints to the model
# conlist2=[0]*3
# conlist2[0] = Mod2.addConstr(ModX2[0] + ModX2[1] + ModX2[2] + ModX2[3] + ModX2[4] + ModX2[5] + ModX2[6] + ModX2[7] +ModX2[8] + ModX2[9] <= 10) 
# conlist2[1] = Mod2.addConstr(1*ModX2[0] + 1*ModX2[1] - 1*ModX2[4] - 1* ModX2[9] <= 0)
# conlist2[2] = Mod2.addConstr(-2*ModX2[2] + -2*ModX2[3] + 1* ModX2[4] + 1*ModX2[5] + 1*ModX2[6] + 1*ModX2[7] + 1*ModX2[8] >= 0)
# Mod2.setObjective(ROI2 @ ModX2, sense=gp.GRB.MAXIMIZE)


# Mod2.Params.OutputFlag = 0 # tell gurobi to shut up!!
# Mod2.Params.TimeLimit = 3600 # stop if taking too long
# Mod2.optimize() # solve the LP, will take the most time
# print(Mod2.objVal) # optimal ROI
# print(Mod2.x) # how much to invest in each investment

10.465
[5.0, 0.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0]


Q5 c) Since the resulting objective income is still the same without the third constraint, we can conclude the third constraint is not helpful. 

#Q6

In [ ]:
ModX.SAObjUp

array([1.049, 1.062, 1.039, 1.046, 1.029, 1.039,   inf, 1.039, 1.039,
         inf])

In [ ]:
ModX.SAObjLow

array([ -inf, 1.039,  -inf, 1.033,  -inf,  -inf, 1.039,  -inf,  -inf,
       1.029])

Q6 a) From the results we can see, if we want the allocation to stay the same using the first ROI:

*   The ROI for print can go up to 4.9% and go down to negative infinity.
*   The ROI for TV can go up to 6.2% and go down to 3.9%. 
*   The ROI for SEO can go up to 3.9% and go down to negative infinity.
*   The ROI for AdWords can go up to 4.6% and go down to 3.3.
*   The ROI for Facebook can go up to 2.9% and go down to negative infinity.
*   The ROI for LinkedIn can go up to 3.9% and go down to negative infinity.
*   The ROI for Instagram can go up to infinity and go down to 3.9%.
*   The ROI for Snapchat can go up to 3.9% and go down to negative infinity.
*   The ROI for Twitter can go up to 3.9% and go down to negative infinity.
*   The ROI for Email can go up to infinity and go down to 2.9%.

In [ ]:
ModX2.SAObjUp

array([1.052, 1.049, 1.039, 1.046,   inf,   inf, 1.039, 1.039, 1.039,
       1.029])

In [ ]:
ModX2.SAObjLow

array([1.039 ,   -inf,   -inf, 1.0375, 1.029 , 1.039 ,   -inf,   -inf,
         -inf,   -inf])

Q6 b) From the results we can see, if we want the allocation to stay the same using the second ROI:

*   The ROI for print can go up to 5.2% and go down to 3.9%.
*   The ROI for TV can go up to 4.9% and go down to negative infinity. 
*   The ROI for SEO can go up to 3.9% and go down to negative infinity.
*   The ROI for AdWords can go up to 4.6% and go down to 3.75%.
*   The ROI for Facebook can go up to infinity and go down to 2.9%.
*   The ROI for LinkedIn can go up to infinity and go down to 3.9.
*   The ROI for Instagram can go up to 3.9 and go down to negative infinity.
*   The ROI for Snapchat can go up to 3.9% and go down to negative infinity.
*   The ROI for Twitter can go up to 3.9% and go down to negative infinity.
*   The ROI for Email can go up to 2.9 and go down to negative infinity.

#Q7-8

In [ ]:
Mod3=gp.Model()
ModX3 = Mod3.addMVar(10, ub = 3) # tell the model how many variables there are

# add the constraints to the model
conlist3=[0]*3
conlist3[0] = Mod3.addConstr(1*ModX3[0] + 1*ModX3[1] - 1*ModX3[4] - 1* ModX3[9] <= 0)
conlist3[1] = Mod3.addConstr(-2*ModX3[2] + -2*ModX3[3] + 1* ModX3[4] + 1*ModX3[5] + 1*ModX3[6] + 1*ModX3[7] + 1*ModX3[8] >= 0)

In [ ]:
#Create a matrix for the objective function of each month
A = np.zeros((12,10))
for index1 in range(len(A)):
  ROI3 = np.array(df2.loc[index1])
  for index2 in range(len(ROI3)):
      ROI3[index2] = ROI3[index2]/100+1
  A[index1] = ROI3


#Create a for loop to find the optimal allocation based on each month's ROI
#Define the constrint regarding the total amount we can invest (based on 'money' variable) for the first month
money = 10
conlist3[2] = Mod3.addConstr(ModX3[0] + ModX3[1] + ModX3[2] + ModX3[3] + ModX3[4] + ModX3[5] + ModX3[6] + ModX3[7] +ModX3[8] + ModX3[9] <= money)
monthCount = 1
for month in A:
  #Update the constraint regarding the total amount we can invest for the current month
  conlist3[2].rhs = money
  Mod3.setObjective(month @ ModX3, sense=gp.GRB.MAXIMIZE)
  Mod3.Params.OutputFlag = 0 # tell gurobi to shut up!!
  Mod3.optimize()
  print(f"Month {monthCount}'s allocation is {Mod3.X}")
  print(f"Month {monthCount}'s income is {Mod3.objVal}")
  print(f"Month {monthCount +1}'s lowest ROI for allocation to stay the same: {Mod3.SAObjLow}")
  print(f"Month {monthCount +1}'s highest ROI for allocation to stay the same: {Mod3.SAObjUp}")
  #Find the amount of money we can invest for the next month
  # money = money + (Mod3.objVal-money)*0.5
  money = 10 + (Mod3.objVal-money)*0.5
  monthCount += 1

Month 1's allocation is [3.0, 0.0, 0.0, 1.3333333333333333, 0.0, 0.0, 2.666666666666667, 0.0, 0.0, 3.0]
Month 1's income is 10.373
Month 2's lowest ROI for allocation to stay the same: [1.039, -inf, -inf, 1.036, -inf, -inf, 1.035, -inf, -inf, 1.0339999999999998]
Month 2's highest ROI for allocation to stay the same: [1.043, 1.04, 1.039, 1.0405000000000002, 1.033, 1.036, 1.03675, 1.036, 1.036, inf]
Month 2's allocation is [3.0, 0.0, 0.0, 2.3954999999999993, 3.0, 0.0, 0.0, 0.0, 1.7909999999999993, 0.0]
Month 2's income is 10.592795999999998
Month 3's lowest ROI for allocation to stay the same: [1.0396666666666665, -inf, -inf, 1.0370000000000001, 1.0346666666666666, -inf, -inf, -inf, 1.0365, 1.0346666666666664]
Month 3's highest ROI for allocation to stay the same: [inf, 1.0396666666666665, 1.038, 1.0385000000000002, inf, 1.037, 1.037, 1.037, 1.03725, 1.0356666666666665]
Month 3's allocation is [0.0, 0.0, 0.0, 3.0, 0.0, 3.0, 1.2031479999999988, 0.0, 3.0, 0.0]
Month 3's income is 10.610664